In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [5]:
data = pd.read_csv("Qmerge1-FirstLastHRV.csv") 
data.head()


,nsrrid,gender_x,race_x,age_s1_x,ang_date,chd_dthdt,chf_date,cvd_dthdt,mi_date,stk_date,...,ihr,NN_RR,AVNN,SDNN,VLF,LF,LF_n,HF,HF_n,LF_HF
0,200079,2,3,56,NaN,NaN,NaN,NaN,NaN,NaN,...,75.112951,0.992000,798.797,46.2118,695.6390,285.530,0.256639,827.0460,0.743361,0.345241
1,200079,2,3,56,NaN,NaN,NaN,NaN,NaN,NaN,...,70.633025,0.968839,849.461,72.6343,1234.8283,1666.850,0.507907,1614.9500,0.492093,1.032140
2,200082,1,1,40,NaN,NaN,NaN,NaN,NaN,NaN,...,59.355400,0.996610,1010.860,39.4332,739.2121,531.024,0.686206,242.8310,0.313794,2.186800
3,200082,1,1,40,NaN,NaN,NaN,NaN,NaN,NaN,...,56.666320,0.996466,1058.830,146.8230,13231.3000,3964.450,0.378345,6513.9400,0.621655,0.608610
4,200108,2,1,60,NaN,NaN,NaN,NaN,NaN,NaN,...,64.954397,0.990741,923.725,27.7213,532.0790,168.406,0.672796,81.9015,0.327204,2.056200


## Column Names description
1. nsrrid = This subject identifier was created by the National Sleep Research Resource (NSRR) team to more easily match with file downloads. Use 'pptid' to link with Biologic Specimen and Data Repository Information Coordinating Center (BioLINCC) and cohort-specific data.

2. BLUE25 = Quality of Life (QOL) (Sleep Heart Health Study Visit One (SHHS1)): Felt downhearted and blue. 
        1: All of the time
        2: Most of the time
        3: A good bit of the time
        4: Some of the time
        5: A little of time
        6: None of the time

3. ihr = Instantaneous heart rate

4. NN_RR = Ratio of consecutive normal sinus beats (NN) over all cardiac inter-beat (RR) intervals

5. AVNN = Mean of all normal sinus to normal sinus interbeat intervals (NN) 

6. SDNN = Standard deviation of all normal sinus to normal sinus interbeat (NN) intervals

7. VLF = Very low frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.003 and 0.04 Hz

8. LF = Low frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.04 and 0.15 Hz

9. HF = High frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.15 and 0.4 Hz

10. HF_n = High frequency power (normalized)

11. LF_HF = The ratio of low to high frequency power

In [6]:
heart_rate_data = data.copy()

In [7]:
heart_rate_data = heart_rate_data[['nsrrid', 'gender_x', 'race_x', 'age_s1_x', 'BLUE25', 'ihr', 'NN_RR', 'AVNN', 'SDNN', 'VLF', 'LF', 'HF', 'HF_n', 'LF_HF']]

In [8]:
heart_rate_data.head()
heart_rate_data.shape[0]

972

In [9]:
# Skipping every other row, to reemove duplicate rows
heart_rate_data = heart_rate_data[::2]
heart_rate_data.head()
heart_rate_data.shape[0] # row count

486

In [10]:
# Renaming columns 
heart_rate_data = heart_rate_data.rename(index=str, columns={'gender_x': 'Gender', 'race_x': 'Race', 'age_s1_x': 'Age'})
heart_rate_data = heart_rate_data[heart_rate_data['BLUE25'] != 'NAN']
heart_rate_data.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,nsrrid,Gender,Race,Age,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,5.0,75.112951,0.992000,798.797,46.21180,695.6390,285.53000,827.04600,0.743361,0.345241
2,200082,1,1,40,6.0,59.355400,0.996610,1010.860,39.43320,739.2121,531.02400,242.83100,0.313794,2.186800
4,200108,2,1,60,5.0,64.954397,0.990741,923.725,27.72130,532.0790,168.40600,81.90150,0.327204,2.056200
6,200109,2,1,71,6.0,76.865428,0.997389,780.585,8.41453,55.3561,8.87934,4.38036,0.330351,2.027080
8,200112,1,1,48,5.0,62.348286,0.996785,962.336,88.53440,3987.3524,3269.69000,292.34900,0.082073,11.184200


In [11]:
dep_data = heart_rate_data.dropna(axis=0, how='any')
dep_data.shape[0]

dep_data.loc[dep_data['BLUE25'] < 4, 'BLUE25'] = 0
dep_data.loc[dep_data['BLUE25'] >= 4, 'BLUE25'] = 1
dep_data.head(2)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,nsrrid,Gender,Race,Age,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,1.0,75.112951,0.99200,798.797,46.2118,695.6390,285.530,827.046,0.743361,0.345241
2,200082,1,1,40,1.0,59.355400,0.99661,1010.860,39.4332,739.2121,531.024,242.831,0.313794,2.186800


In [12]:
dep_data.dtypes

nsrrid      int64
Gender      int64
Race        int64
Age         int64
BLUE25    float64
ihr       float64
NN_RR     float64
AVNN      float64
SDNN      float64
VLF       float64
LF        float64
HF        float64
HF_n      float64
LF_HF     float64
dtype: object

In [13]:
heart_data = dep_data[['BLUE25', 'ihr', 'NN_RR', 'AVNN', 'SDNN', 'VLF', 'LF', 'HF', 'HF_n', 'LF_HF']]

In [14]:
X = heart_data.drop('BLUE25', axis = 1)  #X = df1.drop('ql209f', axis = 1)
y = heart_data['BLUE25']
y_new = dep_data['BLUE25'].unique()


In [15]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")
X_test.head()

Training Data Score: 0.9821428571428571
Testing Data Score: 0.9557522123893806


,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
652,64.508291,0.996894,930.113,49.6384,1092.66050,692.828,780.8260,0.529857,0.887301
624,69.625196,0.997118,861.757,30.3586,341.33360,409.583,165.0230,0.287193,2.481980
378,66.864660,0.997006,897.335,31.9392,577.38100,330.938,118.2970,0.263330,2.797520
62,73.294886,0.997268,818.611,30.2603,448.75243,292.491,172.6830,0.371222,1.693800
466,70.593384,0.997159,849.938,20.2343,185.56210,118.909,88.7831,0.427475,1.339320


In [19]:
predictions = classifier.predict(X_test)

In [20]:
new_data = [00.508291, -0.996894, -930.113, -49.6384, -1092.66050, 692.828, 780.8260, 0.529857, 0.887301]
predictions = classifier.predict([new_data])
predictions

array([1.])